# 제품 이상여부 판별 프로젝트


### 데이터 읽어오기


In [1]:
import os
from pprint import pprint

import optuna
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

/home/elicer/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

RANDOM_STATE = 110

train_data = pd.read_csv("train_data_0827.csv")
test_data = pd.read_csv("test_data_0827.csv")

In [3]:
# dam, fill1, fill2 공통 변수
var_dam_fill = [
    'Receip_No_encoded',
    'Equipment_same_num',
    'PalletID_Collect_Result_encoded',
    'Production_Qty_Collect_Result',
    'WorkMode Collect Result'
]

In [4]:
# 전체 공통 변수
### train
var_all_train = [
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

### test
var_all_test = [
    'Set ID',
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

In [5]:
# '_Dam'을 포함하는 변수 선택
dam_variables = [var for var in train_data.columns if '_Dam' in var]

# train
final_columns_train = var_dam_fill + var_all_train + dam_variables
train_data_dam = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + dam_variables
test_data_dam = test_data[final_columns_test]

In [6]:
# '_Fill1'을 포함하는 변수 선택
fill1_variables = [var for var in train_data.columns if '_Fill1' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill1_variables
train_data_fill1 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill1_variables
test_data_fill1 = test_data[final_columns_test]

In [7]:
# '_Fill2'을 포함하는 변수 선택
fill2_variables = [var for var in train_data.columns if '_Fill2' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill2_variables
train_data_fill2 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill2_variables
test_data_fill2 = test_data[final_columns_test]

In [8]:
# '_AutoClave'을 포함하는 변수 선택
autoclave_variables = [var for var in train_data.columns if '_AutoClave' in var]

# train
final_columns_train = var_all_train + autoclave_variables
train_data_autoclave = train_data[final_columns_train]

# test 
final_columns_test = var_all_test + autoclave_variables
test_data_autoclave = test_data[final_columns_test]

In [9]:
# 각 DataFrame의 칼럼 수 계산
num_columns_train_data = train_data.shape[1]
num_columns_train_data_dam = train_data_dam.shape[1]
num_columns_train_data_autoclave = train_data_autoclave.shape[1]
num_columns_train_data_fill1 = train_data_fill1.shape[1]
num_columns_train_data_fill2 = train_data_fill2.shape[1]

num_columns_test_data = test_data.shape[1]
num_columns_test_data_dam = test_data_dam.shape[1]
num_columns_test_data_autoclave = test_data_autoclave.shape[1]
num_columns_test_data_fill1 = test_data_fill1.shape[1]
num_columns_test_data_fill2 = test_data_fill2.shape[1]

# 각 DataFrame의 칼럼 수 출력
print("----train data-----")
print(f"train_data DataFrame의 칼럼 수: {num_columns_train_data}")
print(f"train_data_dam DataFrame의 칼럼 수: {num_columns_train_data_dam}")
print(f"train_data_autoclave DataFrame의 칼럼 수: {num_columns_train_data_autoclave}")
print(f"train_data_fill1 DataFrame의 칼럼 수: {num_columns_train_data_fill1}")
print(f"train_data_fill2 DataFrame의 칼럼 수: {num_columns_train_data_fill2}")
print("----test data-----")
print(f"test_data DataFrame의 칼럼 수: {num_columns_test_data}")
print(f"test_data_dam DataFrame의 칼럼 수: {num_columns_test_data_dam}")
print(f"test_data_autoclave DataFrame의 칼럼 수: {num_columns_test_data_autoclave}")
print(f"test_data_fill1 DataFrame의 칼럼 수: {num_columns_test_data_fill1}")
print(f"test_data_fill2 DataFrame의 칼럼 수: {num_columns_test_data_fill2}")

----train data-----
train_data DataFrame의 칼럼 수: 40
train_data_dam DataFrame의 칼럼 수: 23
train_data_autoclave DataFrame의 칼럼 수: 8
train_data_fill1 DataFrame의 칼럼 수: 14
train_data_fill2 DataFrame의 칼럼 수: 14
----test data-----
test_data DataFrame의 칼럼 수: 41
test_data_dam DataFrame의 칼럼 수: 24
test_data_autoclave DataFrame의 칼럼 수: 9
test_data_fill1 DataFrame의 칼럼 수: 15
test_data_fill2 DataFrame의 칼럼 수: 15


---

## Optuna

스레스홀드 0.3으로 맞춘상태에서 튜닝 진행한 것

In [10]:
RANDOM_STATE = 110
THRESHOLD = 0.3

In [11]:
import optuna
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

def objectiveCatBoost(trial, x_tr, y_tr, x_val, y_val):
    
    # 'Normal'과 'AbNormal'을 숫자로 변환
    y_tr = y_tr.map({'Normal': 0, 'AbNormal': 1})
    y_val = y_val.map({'Normal': 0, 'AbNormal': 1})

    param = {
        'iterations': trial.suggest_int('iterations', 800, 5000),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.2),
        'depth': trial.suggest_int('depth', 4, 13),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 5),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10, log=True),
        'random_strength': trial.suggest_float('random_strength', 0, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.1, 10),
        'border_count': trial.suggest_int('border_count', 128, 300),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 10),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
        
        'random_seed': RANDOM_STATE,
        'eval_metric': 'F1',
        'logging_level': 'Silent',
        'boosting_type': 'Plain'
    }
    
    model = CatBoostClassifier(**param)
    model.fit(x_tr, y_tr)
    pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    pred = (pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측
    
    score = f1_score(y_val, pred, average="binary")
    
    return score

# 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    train_data.drop("target", axis=1),  
    train_data["target"],               
    test_size=0.2,
    shuffle=True,
    random_state=RANDOM_STATE,
)

# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
study.optimize(lambda trial: objectiveCatBoost(trial, x_train, y_train, x_val, y_val), n_trials=500)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

[I 2024-08-27 16:23:44,319] A new study created in memory with name: no-name-7e156289-746e-4856-b837-d1c9f3a0f94c
[I 2024-08-27 16:23:59,391] Trial 0 finished with value: 0.22463768115942032 and parameters: {'iterations': 1287, 'learning_rate': 0.13203262096748453, 'depth': 7, 'min_data_in_leaf': 4, 'l2_leaf_reg': 4.780689663026719, 'random_strength': 6.824631292225899, 'bagging_temperature': 9.554598960277284, 'border_count': 238, 'scale_pos_weight': 5.61649246971856, 'grow_policy': 'Depthwise'}. Best is trial 0 with value: 0.22463768115942032.
[I 2024-08-27 16:24:09,429] Trial 1 finished with value: 0.2197802197802198 and parameters: {'iterations': 1368, 'learning_rate': 0.0030110113610709835, 'depth': 8, 'min_data_in_leaf': 3, 'l2_leaf_reg': 1.0505036749294598, 'random_strength': 5.595100695350062, 'bagging_temperature': 0.8684052021612866, 'border_count': 198, 'scale_pos_weight': 3.8269447050797387, 'grow_policy': 'SymmetricTree'}. Best is trial 0 with value: 0.22463768115942032.
[

[I 2024-08-27 16:56:54,424] Trial 19 finished with value: 0.2121684867394696 and parameters: {'iterations': 4719, 'learning_rate': 0.09758867718691291, 'depth': 12, 'min_data_in_leaf': 1, 'l2_leaf_reg': 3.7370376286336344, 'random_strength': 2.175665029433702, 'bagging_temperature': 0.6012680873077576, 'border_count': 284, 'scale_pos_weight': 1.0674410565053178, 'grow_policy': 'SymmetricTree'}. Best is trial 15 with value: 0.23127463863337713.
[I 2024-08-27 17:00:18,616] Trial 20 finished with value: 0.2222222222222222 and parameters: {'iterations': 3562, 'learning_rate': 0.053526595531264765, 'depth': 12, 'min_data_in_leaf': 2, 'l2_leaf_reg': 7.481413232538302, 'random_strength': 0.9911188121443996, 'bagging_temperature': 0.10828299341959258, 'border_count': 251, 'scale_pos_weight': 6.59149106322385, 'grow_policy': 'SymmetricTree'}. Best is trial 15 with value: 0.23127463863337713.
[I 2024-08-27 17:02:03,502] Trial 21 finished with value: 0.22733812949640286 and parameters: {'iteratio

[I 2024-08-27 17:32:51,143] Trial 38 finished with value: 0.23303834808259588 and parameters: {'iterations': 2608, 'learning_rate': 0.015852652643708193, 'depth': 13, 'min_data_in_leaf': 5, 'l2_leaf_reg': 1.1977538589999803, 'random_strength': 0.5542711178402808, 'bagging_temperature': 3.3435156358003844, 'border_count': 244, 'scale_pos_weight': 2.570526472348657, 'grow_policy': 'Depthwise'}. Best is trial 23 with value: 0.23809523809523808.
[I 2024-08-27 17:34:00,861] Trial 39 finished with value: 0.23738062755798087 and parameters: {'iterations': 2519, 'learning_rate': 0.013423575069977704, 'depth': 11, 'min_data_in_leaf': 5, 'l2_leaf_reg': 2.0700523506585675, 'random_strength': 4.0687045842651575, 'bagging_temperature': 3.5120763997019333, 'border_count': 244, 'scale_pos_weight': 2.764111873681697, 'grow_policy': 'Depthwise'}. Best is trial 23 with value: 0.23809523809523808.
[I 2024-08-27 17:35:01,126] Trial 40 finished with value: 0.22571428571428573 and parameters: {'iterations':

[I 2024-08-27 17:48:36,208] Trial 57 finished with value: 0.2332361516034985 and parameters: {'iterations': 1982, 'learning_rate': 0.17718295892962332, 'depth': 10, 'min_data_in_leaf': 4, 'l2_leaf_reg': 1.5801815325645934, 'random_strength': 6.136749247260928, 'bagging_temperature': 9.883368461420794, 'border_count': 182, 'scale_pos_weight': 2.8139932654435995, 'grow_policy': 'Depthwise'}. Best is trial 53 with value: 0.24313725490196078.
[I 2024-08-27 17:49:11,066] Trial 58 finished with value: 0.2303206997084548 and parameters: {'iterations': 1976, 'learning_rate': 0.15881570419231786, 'depth': 10, 'min_data_in_leaf': 3, 'l2_leaf_reg': 1.3784483348035996, 'random_strength': 6.29245188626566, 'bagging_temperature': 9.998509668872495, 'border_count': 186, 'scale_pos_weight': 1.9548867482447703, 'grow_policy': 'Depthwise'}. Best is trial 53 with value: 0.24313725490196078.
[I 2024-08-27 17:49:29,979] Trial 59 finished with value: 0.22720897615708274 and parameters: {'iterations': 1714, 

[I 2024-08-27 18:02:10,390] Trial 76 finished with value: 0.2323529411764706 and parameters: {'iterations': 3009, 'learning_rate': 0.18420394192054626, 'depth': 11, 'min_data_in_leaf': 4, 'l2_leaf_reg': 1.0607854971936905, 'random_strength': 9.359198366145588, 'bagging_temperature': 6.724566474301709, 'border_count': 134, 'scale_pos_weight': 2.1102551123877404, 'grow_policy': 'Depthwise'}. Best is trial 53 with value: 0.24313725490196078.
[I 2024-08-27 18:02:43,256] Trial 77 finished with value: 0.2193211488250653 and parameters: {'iterations': 2847, 'learning_rate': 0.022161727195238734, 'depth': 12, 'min_data_in_leaf': 3, 'l2_leaf_reg': 1.135557928361943, 'random_strength': 7.685094026573509, 'bagging_temperature': 7.186938983962753, 'border_count': 158, 'scale_pos_weight': 2.3886307892630025, 'grow_policy': 'Lossguide'}. Best is trial 53 with value: 0.24313725490196078.
[I 2024-08-27 18:03:17,510] Trial 78 finished with value: 0.23087621696801108 and parameters: {'iterations': 2216,

...

[I 2024-08-27 21:22:26,575] Trial 437 finished with value: 0.24802110817941955 and parameters: {'iterations': 1583, 'learning_rate': 0.011093546868732148, 'depth': 9, 'min_data_in_leaf': 4, 'l2_leaf_reg': 2.046244245532402, 'random_strength': 1.1915996508421867, 'bagging_temperature': 7.927622803227754, 'border_count': 200, 'scale_pos_weight': 2.1676394509499777, 'grow_policy': 'Depthwise'}. Best is trial 294 with value: 0.2540880503144654.
[I 2024-08-27 21:22:52,754] Trial 438 finished with value: 0.2365930599369085 and parameters: {'iterations': 1573, 'learning_rate': 0.0075139188638014935, 'depth': 9, 'min_data_in_leaf': 4, 'l2_leaf_reg': 2.098374794323526, 'random_strength': 1.6263367333993508, 'bagging_temperature': 7.77583826694708, 'border_count': 210, 'scale_pos_weight': 1.7116741461521585, 'grow_policy': 'Depthwise'}. Best is trial 294 with value: 0.2540880503144654.
[I 2024-08-27 21:23:18,858] Trial 439 finished with value: 0.2289348171701113 and parameters: {'iterations': 15

[I 2024-08-27 21:31:30,674] Trial 456 finished with value: 0.2056338028169014 and parameters: {'iterations': 1343, 'learning_rate': 0.0012562557901038638, 'depth': 9, 'min_data_in_leaf': 4, 'l2_leaf_reg': 1.994237494778404, 'random_strength': 1.6025815782223216, 'bagging_temperature': 8.563187857230893, 'border_count': 205, 'scale_pos_weight': 2.475438716990256, 'grow_policy': 'Depthwise'}. Best is trial 445 with value: 0.25439783491204326.
[I 2024-08-27 21:31:56,633] Trial 457 finished with value: 0.2301369863013699 and parameters: {'iterations': 1474, 'learning_rate': 0.00701463182084154, 'depth': 9, 'min_data_in_leaf': 4, 'l2_leaf_reg': 2.1402563645240833, 'random_strength': 0.716197293369808, 'bagging_temperature': 8.263012837839726, 'border_count': 208, 'scale_pos_weight': 2.0964471051316522, 'grow_policy': 'Depthwise'}. Best is trial 445 with value: 0.25439783491204326.
[I 2024-08-27 21:33:03,191] Trial 458 finished with value: 0.23927178153446033 and parameters: {'iterations': 3

[I 2024-08-27 21:41:47,159] Trial 475 finished with value: 0.22879177377892032 and parameters: {'iterations': 1678, 'learning_rate': 0.01098332178620216, 'depth': 8, 'min_data_in_leaf': 4, 'l2_leaf_reg': 1.1660386607806963, 'random_strength': 0.8010921289728666, 'bagging_temperature': 8.308568890009365, 'border_count': 203, 'scale_pos_weight': 2.263383040111784, 'grow_policy': 'Depthwise'}. Best is trial 445 with value: 0.25439783491204326.
[I 2024-08-27 21:42:13,314] Trial 476 finished with value: 0.23249299719887956 and parameters: {'iterations': 1328, 'learning_rate': 0.017164266260144008, 'depth': 9, 'min_data_in_leaf': 4, 'l2_leaf_reg': 1.2921101392056547, 'random_strength': 0.42725170763025827, 'bagging_temperature': 7.469263354526021, 'border_count': 212, 'scale_pos_weight': 1.9786097150119706, 'grow_policy': 'Depthwise'}. Best is trial 445 with value: 0.25439783491204326.
[I 2024-08-27 21:42:33,403] Trial 477 finished with value: 0.24203821656050958 and parameters: {'iterations

[I 2024-08-27 21:50:17,126] Trial 494 finished with value: 0.2305389221556886 and parameters: {'iterations': 1587, 'learning_rate': 0.13439983724476517, 'depth': 9, 'min_data_in_leaf': 4, 'l2_leaf_reg': 1.1969584187016609, 'random_strength': 1.467383087383223, 'bagging_temperature': 7.632585287058977, 'border_count': 212, 'scale_pos_weight': 1.920515310538692, 'grow_policy': 'Depthwise'}. Best is trial 445 with value: 0.25439783491204326.
[I 2024-08-27 21:50:48,379] Trial 495 finished with value: 0.23458646616541354 and parameters: {'iterations': 1832, 'learning_rate': 0.00324049483923918, 'depth': 9, 'min_data_in_leaf': 4, 'l2_leaf_reg': 1.2899210593368493, 'random_strength': 1.194152012828178, 'bagging_temperature': 7.92764605280196, 'border_count': 219, 'scale_pos_weight': 2.07426719138714, 'grow_policy': 'Depthwise'}. Best is trial 445 with value: 0.25439783491204326.
[I 2024-08-27 21:51:16,146] Trial 496 finished with value: 0.2 and parameters: {'iterations': 1737, 'learning_rate'

Best trial: score 0.25439783491204326, 
params {'iterations': 1349, 'learning_rate': 0.012526639112437014, 'depth': 9, 'min_data_in_leaf': 4, 'l2_leaf_reg': 2.245006704049574, 'random_strength': 0.6922797458293842, 'bagging_temperature': 8.230635636022027, 'border_count': 211, 'scale_pos_weight': 2.0709015241138236, 'grow_policy': 'Depthwise'}


1. 0.25439783491204326

params {'iterations': 1349, 'learning_rate': 0.012526639112437014, 'depth': 9, 'min_data_in_leaf': 4, 'l2_leaf_reg': 2.245006704049574, 'random_strength': 0.6922797458293842, 'bagging_temperature': 8.230635636022027, 'border_count': 211, 'scale_pos_weight': 2.0709015241138236, 'grow_policy': 'Depthwise'}

In [11]:
# 모델 설정 및 하이퍼파라미터
models = {
    'et': ExtraTreesClassifier(),
    'rf': RandomForestClassifier(),
    'cat': CatBoostClassifier(),
    'lgbm': LGBMClassifier(),
    'xgb': XGBClassifier(),
    'dt': DecisionTreeClassifier()
}

def train_and_evaluate_model(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return
    
    # 데이터셋 분할
    x_train, x_val, y_train, y_val = train_test_split(
        data.drop("target", axis=1),
        data["target"].map({'Normal': 0, 'AbNormal': 1}),
        test_size=0.2,
        shuffle=True,
        random_state=RANDOM_STATE,
    )

    # 모델 선택
    model = models[model_name]

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(x_train, y_train)

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    # 예측
    y_val_pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    y_val_pred = (y_val_pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측

    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 결과 출력
    print(f'{model_name} 모델이 {data_name} 데이터로 학습한 결과:')
    print(f'F1 Score: {f1}')
    print('---')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('---')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print('\n')

In [12]:
train_and_evaluate_model(
    'cat', train_data,
    iterations=1349,
    learning_rate=0.012526639112437014,
    depth=9,
    min_data_in_leaf=4,
    l2_leaf_reg=2.245006704049574,
    random_strength=0.6922797458293842,
    bagging_temperature=8.230635636022027,
    border_count=211,
    scale_pos_weight=2.0709015241138236,
    grow_policy='Depthwise',
    random_seed=RANDOM_STATE,
    eval_metric='F1',
    logging_level='Silent',
    boosting_type='Plain'
)

cat 모델이 train_data 데이터로 학습한 결과:
F1 Score: 0.23880597014925375
---
Confusion Matrix:
[[7453  209]
 [ 352   88]]
---
Accuracy: 0.9307578375709701
Precision: 0.2962962962962963
Recall: 0.2


